In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import os
import geopandas as gpd
from fiona.crs import from_epsg
from geopandas import GeoDataFrame
from shapely.geometry import Point

from functools import partial
import pyproj
from shapely.ops import transform


import matplotlib.pylab as plt
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random

from matplotlib.pylab import plt
%matplotlib inline

In [2]:
jan15_YT = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-01.csv")
jan15_GT = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-01.csv")
jan15_FHV = pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv")

In [3]:
jan15_GT

VendorID lpep_pickup_datetime  \
2 2015-01-01 00:34:42  2015-01-01 00:38:34                    N   
  2015-01-01 00:34:46  2015-01-01 00:47:23                    N   
1 2015-01-01 00:34:44  2015-01-01 00:38:15                    N   
2 2015-01-01 00:34:48  2015-01-01 00:38:08                    N   
  2015-01-01 00:34:53  2015-01-01 01:09:10                    N   
1 2015-01-01 00:34:55  2015-01-01 00:40:58                    N   
  2015-01-01 00:34:49  2015-01-01 00:53:10                    N   
2 2015-01-01 00:35:03  2015-01-01 00:35:08                    N   
  2015-01-01 00:35:13  2015-01-01 00:41:04                    N   
  2015-01-01 00:35:09  2015-01-01 00:42:39                    N   
  2015-01-01 00:34:56  2015-01-01 00:44:39                    N   
  2015-01-01 00:34:59  2015-01-01 00:56:16                    N   
  2015-01-01 00:35:07  2015-01-01 00:57:02                    N   
  2015-01-01 00:35:01  2015-01-01 01:00:07                    N   
1 2015-01-01 00:35:12  2015-01-01 00:41:58                    N   
2 2015-01-01 00:35:14  2015-01-01 00:37:25                    N   
  2015-01-01 00:35:29  2015-01-01 00:38:35                    N   
  2015-01-01 00:35:33  2015-01-01 00:39:43                    N   
  2015-01-01 00:35:13  2015-01-01 00:43:40                    N   
  2015-01-01 00:35:23  2015-01-01 00:43:46                    N   
  2015-01-01 00:35:26  2015-01-01 00:42:53                    N   
  2015-01-01 00:35:40  2015-01-01 00:44:40                    N   
  2015-01-01 00:35:35  2015-01-01 00:52:25                    N   
  2015-01-01 00:35:36  2015-01-01 00:52:35                    N   
  2015-01-01 00:35:41  2015-01-01 00:55:58                    N   
  2015-01-01 00:35:43  2015-01-01 00:56:12                    N   
  2015-01-01 00:35:25  2015-01-01 00:58:06                    N   
  2015-01-01 00:35:44  2015-01-01 01:13:41                    N   
1 2015-01-01 00:35:38  2015-01-01 00:41:46                    N   
  2015-01-01 00:35:21  2015-01-01 01:00:24                    N   
...                                    ...                  ...   
  2015-01-31 23:00:49  2015-01-31 23:11:29                    N   
  2015-01-31 23:01:04  2015-01-31 23:10:17                    N   
  2015-01-31 23:00:43  2015-01-31 23:30:00                    N   
  2015-01-31 23:00:34  2015-01-31 23:03:54                    N   
  2015-01-31 23:00:40  2015-01-31 23:18:45                    N   
  2015-01-31 23:00:26  2015-01-31 23:08:23                    N   
  2015-01-31 23:00:12  2015-01-31 23:06:00                    N   
  2015-01-31 23:01:17  2015-01-31 23:14:37                    N   
  2015-01-31 23:03:53  2015-01-31 23:21:56                    N   
  2015-01-31 23:03:10  2015-01-31 23:15:46                    N   
  2015-01-31 23:02:51  2015-01-31 23:25:02                    N   
  2015-01-31 23:02:40  2015-01-31 23:07:32                    N   
  2015-01-31 23:02:22  2015-01-31 23:12:24                    N   
  2015-01-31 23:02:14  2015-01-31 23:10:28                    N   
  2015-01-31 23:02:28  2015-01-31 23:07:06                    N   
  2015-01-31 23:02:31  2015-01-31 23:24:53                    N   
  2015-01-31 23:01:57  2015-01-31 23:09:23                    N   
  2015-01-31 23:01:53  2015-01-31 23:22:37                    N   
  2015-01-31 23:02:17  2015-01-31 23:12:27                    N   
  2015-01-31 23:01:42  2015-01-31 23:14:08                    N   
  2015-01-31 23:01:10  2015-01-31 23:09:21                    N   
  2015-01-31 23:01:15  2015-01-31 23:04:04                    N   
  2015-01-31 23:00:47  2015-01-31 23:04:45                    N   
  2015-01-31 23:00:23  2015-01-31 23:10:38                    N   
  2015-01-31 23:00:18  2015-01-31 23:17:41                    N   
  2015-01-31 23:00:33  2015-01-31 23:10:15                    N   
  2015-01-31 23:00:09  2015-01-31 23:09:56                    N   
  2015-01-31 23:00:01  2015-01-31 23:29:48                    N   
  2015-01-31 2

In [25]:
!curl -O "https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip" 
!curl -O "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1001k  100 1001k    0     0  1001k      0  0:00:01 --:--:--  0:00:01 5854k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12322  100 12322    0     0  12322      0  0:00:01 --:--:--  0:00:01  106k


In [26]:
os.system("unzip taxi_zones.zip")

0

In [4]:
zones = gpd.read_file("taxi_zones.shp")

In [5]:
weather = pd.read_csv("1106040.csv")

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,15,20,23,25,40,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
jan15_FHV.dropna(axis = 0, inplace=True)

In [7]:
jan15_GT = jan15_GT[(jan15_GT.Pickup_latitude != 0) | (jan15_GT.Pickup_longitude != 0)]
jan15_YT = jan15_YT[(jan15_YT.pickup_latitude != 0) | (jan15_YT.pickup_longitude != 0)]

In [8]:
jan15_GT

VendorID lpep_pickup_datetime  \
2 2015-01-01 00:34:42  2015-01-01 00:38:34                    N   
  2015-01-01 00:34:46  2015-01-01 00:47:23                    N   
1 2015-01-01 00:34:44  2015-01-01 00:38:15                    N   
2 2015-01-01 00:34:48  2015-01-01 00:38:08                    N   
  2015-01-01 00:34:53  2015-01-01 01:09:10                    N   
1 2015-01-01 00:34:55  2015-01-01 00:40:58                    N   
  2015-01-01 00:34:49  2015-01-01 00:53:10                    N   
2 2015-01-01 00:35:03  2015-01-01 00:35:08                    N   
  2015-01-01 00:35:13  2015-01-01 00:41:04                    N   
  2015-01-01 00:35:09  2015-01-01 00:42:39                    N   
  2015-01-01 00:34:56  2015-01-01 00:44:39                    N   
  2015-01-01 00:34:59  2015-01-01 00:56:16                    N   
  2015-01-01 00:35:07  2015-01-01 00:57:02                    N   
  2015-01-01 00:35:01  2015-01-01 01:00:07                    N   
1 2015-01-01 00:35:12  2015-01-01 00:41:58                    N   
2 2015-01-01 00:35:14  2015-01-01 00:37:25                    N   
  2015-01-01 00:35:29  2015-01-01 00:38:35                    N   
  2015-01-01 00:35:33  2015-01-01 00:39:43                    N   
  2015-01-01 00:35:13  2015-01-01 00:43:40                    N   
  2015-01-01 00:35:23  2015-01-01 00:43:46                    N   
  2015-01-01 00:35:26  2015-01-01 00:42:53                    N   
  2015-01-01 00:35:40  2015-01-01 00:44:40                    N   
  2015-01-01 00:35:35  2015-01-01 00:52:25                    N   
  2015-01-01 00:35:36  2015-01-01 00:52:35                    N   
  2015-01-01 00:35:41  2015-01-01 00:55:58                    N   
  2015-01-01 00:35:43  2015-01-01 00:56:12                    N   
  2015-01-01 00:35:25  2015-01-01 00:58:06                    N   
  2015-01-01 00:35:44  2015-01-01 01:13:41                    N   
1 2015-01-01 00:35:38  2015-01-01 00:41:46                    N   
  2015-01-01 00:35:21  2015-01-01 01:00:24                    N   
...                                    ...                  ...   
  2015-01-31 23:00:49  2015-01-31 23:11:29                    N   
  2015-01-31 23:01:04  2015-01-31 23:10:17                    N   
  2015-01-31 23:00:43  2015-01-31 23:30:00                    N   
  2015-01-31 23:00:34  2015-01-31 23:03:54                    N   
  2015-01-31 23:00:40  2015-01-31 23:18:45                    N   
  2015-01-31 23:00:26  2015-01-31 23:08:23                    N   
  2015-01-31 23:00:12  2015-01-31 23:06:00                    N   
  2015-01-31 23:01:17  2015-01-31 23:14:37                    N   
  2015-01-31 23:03:53  2015-01-31 23:21:56                    N   
  2015-01-31 23:03:10  2015-01-31 23:15:46                    N   
  2015-01-31 23:02:51  2015-01-31 23:25:02                    N   
  2015-01-31 23:02:40  2015-01-31 23:07:32                    N   
  2015-01-31 23:02:22  2015-01-31 23:12:24                    N   
  2015-01-31 23:02:14  2015-01-31 23:10:28                    N   
  2015-01-31 23:02:28  2015-01-31 23:07:06                    N   
  2015-01-31 23:02:31  2015-01-31 23:24:53                    N   
  2015-01-31 23:01:57  2015-01-31 23:09:23                    N   
  2015-01-31 23:01:53  2015-01-31 23:22:37                    N   
  2015-01-31 23:02:17  2015-01-31 23:12:27                    N   
  2015-01-31 23:01:42  2015-01-31 23:14:08                    N   
  2015-01-31 23:01:10  2015-01-31 23:09:21                    N   
  2015-01-31 23:01:15  2015-01-31 23:04:04                    N   
  2015-01-31 23:00:47  2015-01-31 23:04:45                    N   
  2015-01-31 23:00:23  2015-01-31 23:10:38                    N   
  2015-01-31 23:00:18  2015-01-31 23:17:41                    N   
  2015-01-31 23:00:33  2015-01-31 23:10:15                    N   
  2015-01-31 23:00:09  2015-01-31 23:09:56                    N   
  2015-01-31 23:00:01  2015-01-31 23:29:48                    N   
  2015-01-31 2

In [9]:
zones = zones.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [10]:
zones = zones.to_crs({'proj':'longlat', "epsg" : "2263"})

In [12]:
geometry_GT = [Pointt(xy) for xy in zip(jan15_GT.Pickup_longitude, jan15_GT.Pickup_latitude)]
crs = {'proj':'longlat', "epsg" : "2263"}
jan15_GT_shp = GeoDataFrame(jan15_GT, crs=crs, geometry=geometry_GT)

In [ ]:
ax1 = zones.plot(figsize=(30,10))
sample = jan15_GT_shp.sample(1000)
sample.plot(ax = ax1, c='r')

In [17]:
geometry_YT = [Point(xy) for xy in zip(jan15_YT.pickup_longitude, jan15_YT.pickup_latitude)]
crs = {'init': 'epsg:2263'}
jan15_YT_shp = GeoDataFrame(jan15_YT, crs=crs, geometry=geometry_YT)

In [18]:
jan15_GT_shp.sample(1000)

VendorID lpep_pickup_datetime  \
2 2015-01-11 10:11:16  2015-01-11 10:23:25                    N   
  2015-01-22 21:02:58  2015-01-22 21:15:47                    N   
  2015-01-24 09:31:29  2015-01-24 09:36:48                    N   
  2015-01-21 21:57:05  2015-01-21 22:03:22                    N   
  2015-01-25 00:56:44  2015-01-25 00:56:49                    N   
  2015-01-15 09:40:33  2015-01-15 09:56:53                    N   
  2015-01-11 16:32:14  2015-01-11 16:46:15                    N   
  2015-01-17 01:12:49  2015-01-17 01:17:20                    N   
  2015-01-12 11:52:37  2015-01-12 12:04:45                    N   
  2015-01-09 13:03:35  2015-01-09 13:20:35                    N   
  2015-01-31 14:57:50  2015-01-31 15:01:53                    N   
  2015-01-23 14:19:33  2015-01-23 14:24:14                    N   
  2015-01-16 12:24:19  2015-01-16 12:38:23                    N   
  2015-01-07 16:04:37  2015-01-07 16:13:39                    N   
1 2015-01-26 15:11:43  2015-01-26 15:21:11                    N   
2 2015-01-07 20:12:50  2015-01-07 20:17:45                    N   
  2015-01-19 09:00:26  2015-01-19 09:17:07                    N   
  2015-01-27 08:38:32  2015-01-27 09:03:52                    N   
1 2015-01-16 09:30:12  2015-01-16 09:56:11                    N   
  2015-01-18 04:27:06  2015-01-18 04:30:46                    N   
2 2015-01-10 16:43:47  2015-01-10 16:46:20                    N   
  2015-01-25 00:34:47  2015-01-25 00:48:02                    N   
  2015-01-10 16:02:57  2015-01-10 16:14:16                    N   
  2015-01-04 01:52:18  2015-01-04 01:57:14                    N   
1 2015-01-15 11:43:03  2015-01-15 11:54:26                    N   
2 2015-01-04 06:15:48  2015-01-04 06:26:51                    N   
  2015-01-12 06:46:28  2015-01-12 06:51:08                    N   
  2015-01-23 16:38:41  2015-01-23 16:46:18                    N   
  2015-01-29 09:19:14  2015-01-29 09:45:14                    N   
  2015-01-11 12:59:27  2015-01-11 13:15:35                    N   
...                                    ...                  ...   
  2015-01-22 17:03:20  2015-01-22 17:15:40                    N   
  2015-01-04 00:26:54  2015-01-04 00:42:49                    N   
  2015-01-08 23:05:10  2015-01-08 23:10:27                    N   
  2015-01-18 17:56:51  2015-01-18 18:01:28                    N   
  2015-01-08 09:50:26  2015-01-08 09:59:05                    N   
1 2015-01-18 10:33:25  2015-01-18 10:50:34                    N   
2 2015-01-13 23:05:40  2015-01-13 23:08:57                    N   
1 2015-01-06 09:54:19  2015-01-06 10:03:40                    N   
2 2015-01-06 22:36:18  2015-01-06 22:46:31                    N   
  2015-01-10 10:29:50  2015-01-10 10:32:46                    N   
  2015-01-09 14:57:01  2015-01-09 15:34:44                    N   
1 2015-01-15 21:50:52  2015-01-15 21:56:07                    N   
2 2015-01-21 10:04:59  2015-01-21 10:08:20                    N   
1 2015-01-02 05:12:21  2015-01-02 05:34:29                    N   
2 2015-01-31 16:20:08  2015-01-31 16:39:46                    N   
  2015-01-30 17:35:44  2015-01-30 17:41:22                    N   
  2015-01-22 18:52:51  2015-01-22 18:59:58                    N   
  2015-01-16 22:14:33  2015-01-16 22:21:48                    N   
1 2015-01-15 20:03:05  2015-01-15 20:11:56                    N   
2 2015-01-12 23:27:55  2015-01-12 23:36:44                    N   
  2015-01-24 04:38:16  2015-01-24 04:56:59                    N   
1 2015-01-22 15:10:45  2015-01-22 15:30:52                    N   
2 2015-01-05 06:19:31  2015-01-05 06:25:36                    N   
  2015-01-19 18:16:19  2015-01-19 18:22:59                    N   
1 2015-01-31 13:40:55  2015-01-31 13:46:44                    N   
2 2015-01-02 12:10:19  2015-01-02 12:37:07                    N   
  2015-01-04 19:46:38  2015-01-04 19:51:52                    N   
1 2015-01-10 04:49:28  2015-01-10 05:03:53                    N   
2 2015-01-16 2

In [24]:
test = jan15_GT_shp.sample(1000)
jan15_GT_loc = gpd.sjoin(test, zones, how='inner', op='intersects')

KeyError: 'index_left'

In [22]:
jan_15_YT_loc = gpd.sjoin(jan15_YT_shp, zones, how="inner", op='intersects')

KeyboardInterrupt: 

In [21]:
jan15_GT_loc

NameError: name 'jan15_GT_loc' is not defined

#TO DO: FIGURE OUT SPATIAL JOIN ISSUE, MAKE SURE TO USE PICKUP LONG LAT, NOT DROP-OFF (THE DATA IS WEIRD SO EVEN IF IT LOOKS LIKE THAT'S WHAT YOU'RE USING YOU'RE NOT)

In [ ]:
jan15_FHV.groupby(["locationID", "Pickup_date"]).count()